In [1]:
import pandas as pd 
import numpy as np 
import math as math
import joblib
from joblib import dump
import os

In [2]:
import pandas as pd 
print(np.__version__)

1.24.4


## LOAD DATA

In [2]:
def load_Dataset(baseFile, fold_Number): 
    rnmColData = ['user_id', 'item_id', 'rating', 'timestamp']
    base_File = f"{baseFile}/u{fold_Number}.base"
    test_File = f"{baseFile}/u{fold_Number}.test"
    base_Data = pd.read_csv(base_File, sep="\t", header=None, names=rnmColData)
    test_Data = pd.read_csv(test_File, sep="\t", header=None, names=rnmColData)
    base_Data = base_Data.drop(columns=["timestamp"])
    test_Data = test_Data.drop(columns=["timestamp"])
    return base_Data, test_Data

In [3]:
call_base = "ml-100k"
basedata, testdata = load_Dataset(call_base, 5)
basedata

,user_id,item_id,rating
0,1,1,5
1,1,2,3
2,1,4,3
3,1,5,3
4,1,6,5
...,...,...,...
79995,943,943,5
79996,943,1011,2
79997,943,1067,2
79998,943,1074,4


## CONVERT RATING MATRIKS

In [4]:
def ConvertRatingMatriks(ratingData):
    # membuat container untuk rating matriks dengan ukuran user x item
    matriks_rating = pd.DataFrame(np.zeros((943, 1682)), columns = list(range(1, 1683)), index = list(range(1, 944)))
    # merubah data frame ke dalam bentuk matriks rating pivot
    convertMatriksRating = ratingData.pivot_table(index = 'user_id', columns = 'item_id', values='rating')
    # mengisi matriks rating NaN dengan 0
    convertMatriksRating = convertMatriksRating.fillna(0)
    # updating matriks rating dengan matriks rating yang sudah di pivot
    matriks_rating.update(convertMatriksRating)
    # mengembalikan matriks rating
    return matriks_rating

## BASE MODEL

In [5]:
rating_Matriks = ConvertRatingMatriks(basedata)
rating_Matriks

,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
1,5.0,3.0,0.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.0,0.0,0.0,2.0,0.0,0.0,0.0,5.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
942,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### MEAN 

In [6]:
def meanRating(RatingMatriks, jenis="user-based"):
    if jenis == "user-based":
        # menghitung berapa banyak rating yang ada pada setiap user(baris)
        axis = 1
        # index baris user dari rating matriks
        index = RatingMatriks.index
    elif jenis == "item-based":
        # menghitung berapa banyak rating yang ada pada setiap item(kolom)
        axis = 0
        # index kolom item dari rating matriks
        index = RatingMatriks.columns
    # menghitung mean rating dari setiap user/item
    pembilang = RatingMatriks.sum(axis=axis)
    # menghitung banyaknya rating yang ada pada setiap user/item yang bukan 0
    penyebut = np.count_nonzero(RatingMatriks, axis=axis)
    # cek jika penyebut == 0, maka mean rating = 0, jika tidak hitung mean rating
    calculateMeanRating = np.where(penyebut == 0, 0, pembilang / penyebut)
    # mengembalikan mean rating dalam bentuk data frame
    meanRatingNew = pd.DataFrame(calculateMeanRating, index=index, columns=["meanRating"])
    return meanRatingNew

#### USER

In [7]:
cal_MeanRatingUser = meanRating(rating_Matriks, jenis="user-based")
cal_MeanRatingUser

,meanRating
1,3.597610
2,3.672414
3,2.745098
4,4.272727
5,2.871166
...,...
939,3.937500
940,3.414634
941,4.428571
942,4.187500


#### ITEM

In [8]:
cal_MeanRatingItem = meanRating(rating_Matriks, jenis="item-based")
cal_MeanRatingItem

,meanRating
1,3.870423
2,3.166667
3,2.986301
4,3.535294
5,3.263889
...,...
1678,1.000000
1679,3.000000
1680,0.000000
1681,0.000000


### MEAN-CENTERED

In [9]:
def meanCenteredRating(RatingMatriks, meanRating, jenis="user-based"):
    # matriks rating jadi numpy array
    npRatingMatriks = np.array(RatingMatriks)
    if jenis == "user-based":
        # Reshape meanRating menjadi (jumlah_user, 1), kolom
        npMeanRating = np.array(meanRating).reshape(-1, 1)  # (943, 1)
    elif jenis == "item-based":
        # Reshape meanRating menjadi (1, jumlah_item), baris
        npMeanRating = np.array(meanRating).reshape(1, -1)  # (1, 1650)
    # Menghitung mean centered, hanya mengurangi rating yang bukan 0, jika rating 0 maka hasilnya 0
    meanCentered = np.where(npRatingMatriks != 0, npRatingMatriks - npMeanRating, 0)
    # Mengembalikan dalam bentuk DataFrame
    dfMeanCentered = pd.DataFrame(meanCentered, index=RatingMatriks.index, columns=RatingMatriks.columns)
    return dfMeanCentered

#### USER

In [10]:
cal_MeanCenteredUser = meanCenteredRating(rating_Matriks, cal_MeanRatingUser, jenis="user-based")
cal_MeanCenteredUser

,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
1,1.402390,-0.597610,0.0,-0.597610,-0.59761,1.40239,0.402390,-2.597610,1.402390,-0.597610,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.327586,0.000000,0.0,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,-1.672414,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.0,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.0,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.128834,0.128834,0.0,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.000000,0.000000,0.0,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.000000,0.000000,0.0,-1.414634,0.00000,0.00000,0.000000,1.585366,-0.414634,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,0.000000,0.000000,0.0,0.000000,0.00000,0.00000,-0.428571,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
942,0.000000,0.000000,0.0,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### ITEM

In [11]:
cal_MeanCenteredItem = meanCenteredRating(rating_Matriks, cal_MeanRatingItem, jenis="item-based")
cal_MeanCenteredItem

,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
1,1.129577,-0.166667,0.0,-0.535294,-0.263889,1.25,0.206897,-3.043011,1.089202,-0.794118,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.129577,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,-1.794118,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.129577,-0.166667,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.000000,0.000000,0.0,-1.535294,0.000000,0.00,0.000000,0.956989,-0.910798,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.206897,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
942,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### SIMILARITY

In [12]:
def SimilarityRJ(RatingMatriks, index1, index2, jenis="user-based"):
    # jika jenis == "user-based":
    if jenis == "user-based":
        Rated1 = set(RatingMatriks.columns[RatingMatriks.iloc[index1, :] != 0])
        # print("itemRated1: ", itemRated1)
        Rated2 = set(RatingMatriks.columns[RatingMatriks.iloc[index2, :] != 0])
        NotRated1 = set(RatingMatriks.columns[RatingMatriks.iloc[index1, :] == 0])
        NotRated2 = set(RatingMatriks.columns[RatingMatriks.iloc[index2, :] == 0])
        # print("Rated2: ", Rated2)
    elif jenis == "item-based":
        # mencari index user yang telah memberi rating pada item1 dan item2
        Rated1 = set(RatingMatriks.index[RatingMatriks.iloc[:, index1] != 0])
        Rated2 = set(RatingMatriks.index[RatingMatriks.iloc[:, index2] != 0])
        NotRated1 = set(RatingMatriks.index[RatingMatriks.iloc[:, index1] == 0])
        NotRated2 = set(RatingMatriks.index[RatingMatriks.iloc[:, index2] == 0])
    # menghitung banyaknya item yang telah diberi rating oleh user1 dan user2
    intersection = len(Rated1.intersection(Rated2))
    #menghitung jumlah total item yang telah diberi rating oleh user dan jumlah total item yang telah diberi rating oleh user
    CountNotRated1 = len(NotRated1)
    CountNotRated2 = len(NotRated2)
    # menghitung similarity
    if intersection != 0:
        rumusRelevantJaccard = 1/(1+(1/intersection)+(CountNotRated1/(1+CountNotRated1))+(1/(1+CountNotRated2)))
    elif intersection == 0:
        rumusRelevantJaccard = 0
    return rumusRelevantJaccard

In [13]:
cal_simRJUser = SimilarityRJ(rating_Matriks, 0, 0, jenis="user-based")
cal_simRJUser

0.4981549815498156

In [14]:
cal_simRJItem = SimilarityRJ(rating_Matriks, 1, 2, jenis="item-based")
cal_simRJItem

0.48889872375808235

In [15]:
def SimilarityAllRJ(RatingMatriks, jenis="user-based"):
    if jenis == "user-based":
        jumlah = RatingMatriks.shape[0]
        simMatriks = np.zeros((jumlah, jumlah))
        for i in range(jumlah):
            for j in range(i, jumlah):
                nilaiSim = SimilarityRJ(RatingMatriks, i, j, jenis=jenis)
                simMatriks[i][j] = nilaiSim
                simMatriks[j][i] = nilaiSim
        simMatriksDf = pd.DataFrame(simMatriks, index=RatingMatriks.index, columns=RatingMatriks.index)
    elif jenis == "item-based":
        jumlah = RatingMatriks.shape[1]
        simMatriks = np.zeros((jumlah, jumlah))
        for i in range(jumlah):
            for j in range(i, jumlah):
                nilaiSim = SimilarityRJ(RatingMatriks, i, j, jenis=jenis)
                simMatriks[i][j] = nilaiSim
                simMatriks[j][i] = nilaiSim
        simMatriksDf = pd.DataFrame(simMatriks, index=RatingMatriks.columns, columns=RatingMatriks.columns)
    return simMatriksDf

In [16]:
calRJALLUser2 = SimilarityAllRJ(rating_Matriks, jenis="user-based")
calRJALLUser2

,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
1,0.498155,0.461546,0.428579,0.400007,0.488376,0.492540,0.492299,0.454554,0.400008,0.488893,...,0.493329,0.466675,0.490195,0.476199,0.486489,0.466674,0.490199,0.466676,0.482764,0.492954
2,0.461546,0.493827,0.428572,0.400002,0.333331,0.481475,0.470572,0.333334,0.428573,0.454541,...,0.466655,0.478261,0.489352,0.483871,0.486480,0.470587,0.479994,0.454547,0.466663,0.466655
3,0.428579,0.428572,0.491228,0.428572,0.333331,0.461532,0.399988,0.428571,0.000000,0.428567,...,0.428561,0.333333,0.473674,0.444444,0.454539,0.399999,0.473677,0.400000,0.444441,0.333327
4,0.400007,0.400002,0.428572,0.482759,0.333330,0.000000,0.428557,0.399999,0.000000,0.333330,...,0.333326,0.333332,0.461528,0.428570,0.444437,0.333332,0.454538,0.428571,0.454540,0.399990
5,0.488376,0.333331,0.333331,0.333330,0.497268,0.484847,0.491513,0.461543,0.400005,0.473684,...,0.492745,0.444448,0.473680,0.400003,0.479998,0.444448,0.485713,0.444450,0.480001,0.492530
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.466674,0.470587,0.399999,0.333332,0.444448,0.470593,0.470603,0.333333,0.000000,0.400003,...,0.466678,0.486486,0.489371,0.476190,0.491531,0.494949,0.454541,0.461541,0.428569,0.479989
940,0.490199,0.479994,0.473677,0.454538,0.485713,0.491804,0.493518,0.461532,0.428565,0.491227,...,0.494512,0.466661,0.490570,0.481475,0.486487,0.454541,0.497674,0.466674,0.489364,0.491222
941,0.466676,0.454547,0.400000,0.428571,0.444450,0.454552,0.400013,0.444445,0.399999,0.333336,...,0.428583,0.461540,0.484860,0.466668,0.482766,0.461541,0.466674,0.488889,0.444440,0.454533
942,0.482764,0.466663,0.444441,0.454540,0.480001,0.485717,0.487818,0.428568,0.399996,0.480001,...,0.490919,0.444441,0.480006,0.461535,0.476193,0.428569,0.489364,0.444440,0.496855,0.487171


In [17]:
calRJALLItem2 = SimilarityAllRJ(rating_Matriks, jenis="item-based")
calRJALLItem2

,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
1,0.499348,0.496790,0.496023,0.497774,0.493662,0.473841,0.498824,0.497858,0.498380,0.494537,...,0.333414,0.0,0.000000,0.000000,0.333414,0.000000,0.000000,0.000000,0.333414,0.333414
2,0.496790,0.497630,0.488899,0.496099,0.490578,0.454568,0.496090,0.493802,0.492479,0.480010,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333348,0.333348
3,0.496023,0.488899,0.496689,0.492719,0.480002,0.444458,0.494847,0.491193,0.494049,0.483872,...,0.000000,0.0,0.000000,0.000000,0.333343,0.000000,0.000000,0.000000,0.000000,0.333343
4,0.497774,0.496099,0.492719,0.498542,0.492994,0.470635,0.497394,0.496598,0.496517,0.489397,...,0.000000,0.0,0.333359,0.333359,0.333359,0.000000,0.000000,0.000000,0.333359,0.333359
5,0.493662,0.490578,0.480002,0.492994,0.496403,0.333340,0.494519,0.488852,0.491158,0.444443,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333342
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.333333,0.333333,0.333333,0.000000,0.000000
1679,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.333333,0.333333,0.333333,0.000000,0.000000
1680,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.333333,0.333333,0.333333,0.000000,0.000000
1681,0.333414,0.333348,0.000000,0.333359,0.000000,0.000000,0.333390,0.333360,0.333375,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000


In [24]:
print(calRJALLItem2.shape)

(1682, 1682)


### TOP-K

In [19]:
#use .pkl to find topK

cal_RJ = joblib.load("case/sim/relevantJ2/ib/simRJFold5.joblib")
cal_RJ

,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
1,0.499297,0.496309,0.495280,0.497562,0.494110,0.476330,0.498662,0.497724,0.498121,0.494253,...,0.333404,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
2,0.496309,0.497561,0.489805,0.495909,0.490206,0.444465,0.495697,0.494086,0.493107,0.482769,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
3,0.495280,0.489805,0.496599,0.493636,0.485715,0.444457,0.495302,0.490525,0.489743,0.480002,...,0.000000,0.0,0.0,0.0,0.333343,0.000000,0.000000,0.0,0.0,0.0
4,0.497562,0.495909,0.493636,0.498534,0.493542,0.454589,0.497411,0.496809,0.496164,0.489398,...,0.000000,0.0,0.0,0.0,0.333359,0.000000,0.000000,0.0,0.0,0.0
5,0.494110,0.490206,0.485715,0.493542,0.496552,0.400010,0.494734,0.488848,0.491750,0.444445,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.333333,0.333333,0.0,0.0,0.0
1679,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.333333,0.333333,0.0,0.0,0.0
1680,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
1681,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0


In [ ]:
# def TopKTetangga(similarity, k):
#     # Mengambil k tetangga terdekat 
#     # Mengurutkan similarity dalam urutan desending
#     return np.argsort(-similarity)[:k]

In [13]:
def TetanggaK(target_user, target_item, rating_matrix_np, similarity_np, k, jenis="user-based"):
    """
    Mengembalikan indeks K tetangga terdekat (user atau item) berdasarkan similarity.
    """
    rating_matrix = rating_matrix_np
    if jenis == "user-based":
        # Ambil semua rating terhadap item target, data rating untuk item target
        item_ratings = rating_matrix[:, target_item]
        tetangga_idx = np.where((item_ratings != 0))[0]
        sim_scores = similarity_np[target_user, tetangga_idx]
        # cek jumalh rating
        martiks = rating_matrix[target_user, tetangga_idx]
        # print(f"matriks rating {martiks.shape}")

    else:  # item-based
        # Di item-based, user tetap user, item adalah target
        user_ratings = rating_matrix_np[target_user, :]
        tetangga_idx = np.where((user_ratings != 0))[0]
        # Ambil skor similarity item-item
        sim_scores = similarity_np[tetangga_idx, target_item]
    # Urutkan tetangga berdasarkan similarity tertinggi
    if len(sim_scores) == 0:
        return []

    # print(f"sim scores {sim_scores}")
    # print(f"sim scores {sim_scores.shape}")
    sorted_idx = np.argsort(-sim_scores)[:k]
    # print(f"sorted idx {sorted_idx}")
    # print(f"sim from soreted {sorted_idx} {sim_scores[sorted_idx]}")
    return tetangga_idx[sorted_idx]



## PREDIKSI

In [14]:
def PrediksiCF(RatingMatriks, similarityFunction, mean, meanCen, user, item, k=2, jenis="user-based"):
    """
    Menghitung prediksi rating menggunakan Collaborative Filtering berbasis user/item, versi optimal.
    """

    # Cache ke bentuk NumPy
    rating_np = RatingMatriks.to_numpy()

    # cek jumlah yang pernah memberikan rating 
    # print(f"jumlah rating item {item} {np.count_nonzero(rating_np[:, item])}")


    similarity_np = similarityFunction.to_numpy()
    # print(similarity_np.shape)
    mean_np = mean.to_numpy()
    # print(mean_np.shape)
    meanCen_np = meanCen.to_numpy()
    # print("mean cen", meanCen_np.shape)

    # Set indeks untuk pencarian tetangga
    target_user = user
    target_item = item
    # print(f"target user {target_user} {target_item}")
    mean_value = mean_np[user] if jenis == "user-based" else mean_np[item]
    # print(f"mean {mean_value}")

    # Cari tetangga
    tetangga = TetanggaK(target_user, target_item, rating_np, similarity_np, k, jenis)
    # print(f"tetangga {tetangga}")
    if len(tetangga) == 0:
        return float(mean_value)

    # Ambil similarity dan mean centered rating
    if jenis == "user-based":
        sim = similarity_np[user, tetangga] # ambil similarity user-user
        # print(f'sim {sim}')
        mean_cen = meanCen_np[tetangga, item] 
        # print(f"mean cen {mean_cen}")
        # cek tetangga dan item pada mean cen
        # print(f"mean cen {meanCen_np[tetangga, item].shape}")
    else:  # item-based
        sim = similarity_np[tetangga, item] # ambil similarity item-item
        # print(f"sim item {sim.shape}") 
        mean_cen = meanCen_np[user, tetangga] # ambil mean centered rating

    # vektor ((a1 * b1) + (a2 * b2))
    pembilang = np.dot(mean_cen, sim)
    # print(f"pembilang {pembilang}")
    penyebut = np.sum(np.abs(sim))
    # print(f"penyebut {penyebut}")

    prediksi = mean_value + (pembilang / penyebut) if penyebut != 0 else mean_value
    # print(f"prediksi {mean_value} + {pembilang} / {penyebut}")
    return float(prediksi)

In [15]:
def prediksi_semua_matriks2(RatingMatriks, similarityFunction, mean, meanCen, k=2, jenis="user-based"):
    """
    Mengisi semua rating kosong dalam matriks dengan hasil prediksi CF.
    """
    # salin rating matriks
    hasil_matriks = RatingMatriks.copy()
    # mendapatkan ukuran matriks rating/ ambil
    num_users, num_items = hasil_matriks.shape
    # loop setiap user
    for user in range(num_users):
        # loop setiap item
        for item in range(num_items):
            if hasil_matriks.iloc[user, item] == 0:
                pred_rating = PrediksiCF(
                    RatingMatriks=RatingMatriks,
                    similarityFunction=similarityFunction,
                    mean=mean,
                    meanCen=meanCen,
                    user=user,
                    item=item,
                    k=k,
                    jenis=jenis # user-based atau item-based
                )
                # isi prediksi hasil ke dalam matriks
                hasil_matriks.iloc[user, item] = pred_rating
    return hasil_matriks

In [16]:
def simpan_semua_prediksi_k(
    RatingMatriks, similarityFunction, mean, meanCen,
    variasi_k, jenis="user-based",
    folder_output="case/prediksi/Jac/ub/5"
):
    for k in variasi_k:
        # Hitung prediksi
        prediksi = prediksi_semua_matriks2(
            RatingMatriks=RatingMatriks,
            similarityFunction=similarityFunction,
            mean=mean,
            meanCen=meanCen,
            k=k,
            jenis=jenis
        )
        # Siapkan path file
        path_file = os.path.join(folder_output, f"{k}.joblib")
        os.makedirs(folder_output, exist_ok=True)

        # Simpan hasil prediksi
        joblib.dump(prediksi, path_file)
        print(f"✔️ Prediksi untuk k={k} disimpan di: {path_file}")

In [22]:
simpan_semua_prediksi_k(
    RatingMatriks=rating_Matriks,
    similarityFunction=cal_RJ,
    mean=cal_MeanRatingItem,
    meanCen=cal_MeanCenteredItem,
    variasi_k=[18, 20, 25],
    jenis="item-based",
    folder_output="case/prediksiModel/RJ/ib/5"
)

✔️ Prediksi untuk k=18 disimpan di: case/prediksiModel/RJ/ib/5\18.joblib
✔️ Prediksi untuk k=20 disimpan di: case/prediksiModel/RJ/ib/5\20.joblib
✔️ Prediksi untuk k=25 disimpan di: case/prediksiModel/RJ/ib/5\25.joblib


# export to joblib

In [22]:
import joblib
import os

def process_and_save_mean_ratings(baseFile, saveDirectory, jenis="user-based"):
    # Loop untuk 5 fold
    for fold_number in range(1, 6):
        # Load dataset untuk base dan test
        base_Data, _ = load_Dataset(baseFile, fold_number)
        
        # Convert ke rating matriks
        matriks_rating = ConvertRatingMatriks(base_Data)
    
        # simlarity jaccard 
        simRJBase = SimilarityAllRJ(matriks_rating,  jenis=jenis)
        simRJFile = f"{saveDirectory}/simRJFold{fold_number}.joblib"
        joblib.dump(simRJBase, simRJFile)
        print(f"[INFO] sim RJ for fold {fold_number} ({jenis}) saved successfully at {simRJFile}")

# Contoh penggunaan
baseFile = "ml-100k"  # Ganti dengan path file data Anda


saveSimUBrj = "case/sim/relevantJ2/ub"
saveSimIBrj = "case/sim/relevantJ2/ib"

process_and_save_mean_ratings(baseFile, saveSimUBrj, jenis="user-based")

# Menyimpan mean centered untuk item-based
process_and_save_mean_ratings(baseFile, saveSimIBrj, jenis="item-based")

[INFO] sim RJ for fold 1 (user-based) saved successfully at case/sim/relevantJ2/ub/simRJFold1.joblib
[INFO] sim RJ for fold 2 (user-based) saved successfully at case/sim/relevantJ2/ub/simRJFold2.joblib
[INFO] sim RJ for fold 3 (user-based) saved successfully at case/sim/relevantJ2/ub/simRJFold3.joblib
[INFO] sim RJ for fold 4 (user-based) saved successfully at case/sim/relevantJ2/ub/simRJFold4.joblib
[INFO] sim RJ for fold 5 (user-based) saved successfully at case/sim/relevantJ2/ub/simRJFold5.joblib
[INFO] sim RJ for fold 1 (item-based) saved successfully at case/sim/relevantJ2/ib/simRJFold1.joblib
[INFO] sim RJ for fold 2 (item-based) saved successfully at case/sim/relevantJ2/ib/simRJFold2.joblib
[INFO] sim RJ for fold 3 (item-based) saved successfully at case/sim/relevantJ2/ib/simRJFold3.joblib
[INFO] sim RJ for fold 4 (item-based) saved successfully at case/sim/relevantJ2/ib/simRJFold4.joblib
[INFO] sim RJ for fold 5 (item-based) saved successfully at case/sim/relevantJ2/ib/simRJFol

In [16]:
dataMatriks = joblib.load("case/ratingMatriks/u1.joblib")
dataMatriks

,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
1,5.0,3.0,4.0,3.0,3.0,0.0,4.0,1.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.0,0.0,0.0,2.0,0.0,0.0,4.0,5.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,5.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
942,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
